In [1]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#import matplotlib.pyplot as plt

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.set_random_seed(777)

In [3]:
x_data = [[1, 2, 1],
          [1, 3, 2],
          [1, 3, 4],
          [1, 5, 5],
          [1, 7, 5],
          [1, 2, 5],
          [1, 6, 6],
          [1, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

# Evaluation our model using this test dataset
x_test = [[2, 1, 1],
          [3, 1, 2],
          [3, 3, 4]]
y_test = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1]]

X = tf.placeholder("float", [None, 3])
Y = tf.placeholder("float", [None, 3])

W = tf.Variable(tf.random_normal([3, 3]))
b = tf.Variable(tf.random_normal([3]))

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
# Try to change learning_rate to small numbers
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Correct prediction Test model
prediction = tf.argmax(hypothesis, 1)
is_correct = tf.equal(prediction, tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(201):
        cost_val, W_val, _ = sess.run([cost, W, optimizer], feed_dict={X: x_data, Y: y_data})
        print(step, cost_val, W_val)

    # predict
    print("Prediction:", sess.run(prediction, feed_dict={X: x_test}))
    # Calculate the accuracy
    print("Accuracy: ", sess.run(accuracy, feed_dict={X: x_test, Y: y_test}))

0 1.1558543 [[ 0.5807229   0.53629667  0.28503624]
 [-0.880366   -0.5175341  -0.27097937]
 [ 1.1865109   0.58328104 -0.3828085 ]]
1 1.0075483 [[ 0.5547206   0.5424017   0.30493358]
 [-0.9547444  -0.49127546 -0.22285959]
 [ 1.0872434   0.63115525 -0.3314152 ]]
2 0.92436826 [[ 0.5528882   0.5286323   0.3205353 ]
 [-0.90297264 -0.5720706  -0.19383623]
 [ 1.1160883   0.56790024 -0.2970051 ]]
3 0.89032674 [[ 0.53469634  0.53762937  0.3297301 ]
 [-0.9394094  -0.5328238  -0.19664629]
 [ 1.0536007   0.62223166 -0.28884894]]
4 0.87250215 [[ 0.5328474   0.52988255  0.33932585]
 [-0.89041984 -0.58170545 -0.19675417]
 [ 1.0803318   0.58640695 -0.27975532]]
5 0.8590492 [[ 0.51894367  0.53647655  0.34663558]
 [-0.90588945 -0.55437213 -0.20861787]
 [ 1.0402896   0.62676066 -0.28006688]]
6 0.8482194 [[ 0.5151779   0.5313864   0.35549155]
 [-0.86864805 -0.58820254 -0.21202885]
 [ 1.0560089   0.6048408  -0.27386636]]
7 0.8391906 [[ 0.5031324   0.5358755   0.36304793]
 [-0.87600094 -0.57094026 -0.2219382

In [4]:
xy = np.array([[828.659973, 833.450012, 908100, 828.349976, 831.659973],
               [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
               [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
               [816, 820.958984, 1008100, 815.48999, 819.23999],
               [819.359985, 823, 1188100, 818.469971, 818.97998],
               [819, 823, 1198100, 816, 820.450012],
               [811.700012, 815.25, 1098100, 809.780029, 813.669983],
               [809.51001, 816.659973, 1398100, 804.539978, 809.559998]])

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

for step in range(101):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

0 Cost:  468727820000.0 
Prediction:
 [[483421.66]
 [973444.06]
 [765719.4 ]
 [536688.44]
 [632562.25]
 [637885.1 ]
 [584626.25]
 [744408.4 ]]
1 Cost:  5.149822e+26 
Prediction:
 [[-1.6007644e+13]
 [-3.2225028e+13]
 [-2.5350266e+13]
 [-1.7770402e+13]
 [-2.0943370e+13]
 [-2.1119646e+13]
 [-1.9356887e+13]
 [-2.4645164e+13]]
2 Cost:  inf 
Prediction:
 [[5.3059493e+20]
 [1.0681420e+21]
 [8.4026881e+20]
 [5.8902393e+20]
 [6.9419618e+20]
 [7.0003903e+20]
 [6.4161004e+20]
 [8.1689710e+20]]
3 Cost:  inf 
Prediction:
 [[-1.7587284e+28]
 [-3.5405008e+28]
 [-2.7851842e+28]
 [-1.9523994e+28]
 [-2.3010068e+28]
 [-2.3203740e+28]
 [-2.1267031e+28]
 [-2.7077157e+28]]
4 Cost:  inf 
Prediction:
 [[5.8295432e+35]
 [1.1735468e+36]
 [9.2318691e+35]
 [6.4714914e+35]
 [7.6269981e+35]
 [7.6911927e+35]
 [7.0492449e+35]
 [8.9750898e+35]]
5 Cost:  inf 
Prediction:
 [[-inf]
 [-inf]
 [-inf]
 [-inf]
 [-inf]
 [-inf]
 [-inf]
 [-inf]]
6 Cost:  nan 
Prediction:
 [[nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]
 [nan]]


In [5]:
def min_max_scaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)
    # noise term prevents the zero division
    return numerator / (denominator + 1e-7)


xy = np.array(
    [
        [828.659973, 833.450012, 908100, 828.349976, 831.659973],
        [823.02002, 828.070007, 1828100, 821.655029, 828.070007],
        [819.929993, 824.400024, 1438100, 818.97998, 824.159973],
        [816, 820.958984, 1008100, 815.48999, 819.23999],
        [819.359985, 823, 1188100, 818.469971, 818.97998],
        [819, 823, 1198100, 816, 820.450012],
        [811.700012, 815.25, 1098100, 809.780029, 813.669983],
        [809.51001, 816.659973, 1398100, 804.539978, 809.559998],
    ]
)

# very important. It does not work without it.
xy = min_max_scaler(xy)
print(xy)

'''
[[0.99999999 0.99999999 0.         1.         1.        ]
 [0.70548491 0.70439552 1.         0.71881782 0.83755791]
 [0.54412549 0.50274824 0.57608696 0.606468   0.6606331 ]
 [0.33890353 0.31368023 0.10869565 0.45989134 0.43800918]
 [0.51436    0.42582389 0.30434783 0.58504805 0.42624401]
 [0.49556179 0.42582389 0.31521739 0.48131134 0.49276137]
 [0.11436064 0.         0.20652174 0.22007776 0.18597238]
 [0.         0.07747099 0.5326087  0.         0.        ]]
'''

x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
train = tf.train.GradientDescentOptimizer(learning_rate=1e-5).minimize(cost)

# Launch the graph in a session.
with tf.Session() as sess:
    # Initializes global variables in the graph.
    sess.run(tf.global_variables_initializer())

    for step in range(101):
        _, cost_val, hy_val = sess.run(
            [train, cost, hypothesis], feed_dict={X: x_data, Y: y_data}
        )
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

[[0.99999999 0.99999999 0.         1.         1.        ]
 [0.70548491 0.70439552 1.         0.71881782 0.83755791]
 [0.54412549 0.50274824 0.57608696 0.606468   0.6606331 ]
 [0.33890353 0.31368023 0.10869565 0.45989134 0.43800918]
 [0.51436    0.42582389 0.30434783 0.58504805 0.42624401]
 [0.49556179 0.42582389 0.31521739 0.48131134 0.49276137]
 [0.11436064 0.         0.20652174 0.22007776 0.18597238]
 [0.         0.07747099 0.5326087  0.         0.        ]]
0 Cost:  1.1256323 
Prediction:
 [[-0.7942449 ]
 [-0.98297   ]
 [-0.4594388 ]
 [ 0.14247596]
 [-0.22507024]
 [-0.16875577]
 [ 0.46886328]
 [ 0.43321937]]
1 Cost:  1.1255717 
Prediction:
 [[-0.7941978 ]
 [-0.982926  ]
 [-0.45940268]
 [ 0.14250308]
 [-0.22503722]
 [-0.16872412]
 [ 0.46888247]
 [ 0.43323776]]
2 Cost:  1.1255114 
Prediction:
 [[-0.7941507 ]
 [-0.9828819 ]
 [-0.45936668]
 [ 0.1425302 ]
 [-0.22500432]
 [-0.16869253]
 [ 0.46890166]
 [ 0.43325615]]
3 Cost:  1.1254508 
Prediction:
 [[-0.7941035 ]
 [-0.9828379 ]
 [-0.45933